#  <center> Just Run all the cells to see latest data </center>

In [1]:
import pandas as pd 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from bs4 import BeautifulSoup
import requests
import re
import json

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

In [2]:
df=pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')

In [3]:
df

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...
118021,2021-05-10,ZW,Zimbabwe,AFRO,5,38419,0,1576
118022,2021-05-11,ZW,Zimbabwe,AFRO,14,38433,0,1576
118023,2021-05-12,ZW,Zimbabwe,AFRO,15,38448,3,1579
118024,2021-05-13,ZW,Zimbabwe,AFRO,18,38466,0,1579


In [4]:
df=df.set_index("Country")

In [5]:
CovidIndia=df.loc["India"]

In [6]:
CovidIndiaPlot=CovidIndia[["Date_reported","New_cases","New_deaths"]]

In [7]:
CovidIndiaPlot

,Date_reported,New_cases,New_deaths
Country,,,
India,2020-01-03,0,0
India,2020-01-04,0,0
India,2020-01-05,0,0
India,2020-01-06,0,0
India,2020-01-07,0,0
...,...,...,...
India,2021-05-10,366161,3754
India,2021-05-11,329942,3876
India,2021-05-12,348421,4205


In [8]:
CovidIndiaPlot=CovidIndiaPlot.reset_index(drop=True)

In [9]:
CovidIndiaPlot["Date_reported"] = pd.to_datetime(CovidIndiaPlot["Date_reported"])

In [10]:
CovidIndiaPlot["MonthYear"]=CovidIndiaPlot["Date_reported"].dt.strftime('%b-%Y')

In [11]:
CovidIndiaPlot=CovidIndiaPlot.set_index("Date_reported")

In [12]:
CovidIndiaPlot.sort_index(inplace=True)

In [13]:
CovidIndiaPlot.reset_index(drop=False,inplace=True)

In [14]:
# plt.figure(figsize=(18,10)) 
# plt.ylabel("daly cases")
# plt.title("Daily Covid data in india")
# plt.plot(CovidIndiaPlot["Date_reported"],CovidIndiaPlot["New_cases"],color="purple")
# p2=plt.twinx()

In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=CovidIndiaPlot["Date_reported"],
                         y=CovidIndiaPlot["New_cases"],
                         mode="lines",
                         line=dict(color='firebrick', width=1.8),
                         name = "Daily cases"
                        )
             )
fig.update_layout(title='Daily New Cases in India',
                  autosize=True,
                  margin=dict(
                        autoexpand=False,
                        l=40,
                        r=20,
                        t=80,
                        ))
fig.update_yaxes(range=[0,450000])
fig.update_xaxes(constraintoward='left')
fig.show()

### Outcome : Sudden spike and low show lack of proper system to gather data
#### -------------------------------------------------------------------------------------------------------------------------------

In [16]:
CovidIndiaPlot

,Date_reported,New_cases,New_deaths,MonthYear
0,2020-01-03,0,0,Jan-2020
1,2020-01-04,0,0,Jan-2020
2,2020-01-05,0,0,Jan-2020
3,2020-01-06,0,0,Jan-2020
4,2020-01-07,0,0,Jan-2020
...,...,...,...,...
493,2021-05-10,366161,3754,May-2021
494,2021-05-11,329942,3876,May-2021
495,2021-05-12,348421,4205,May-2021
496,2021-05-13,362727,4120,May-2021


In [17]:
fig2=go.Figure()
fig2.add_trace(go.Scatter(x=CovidIndiaPlot["Date_reported"],
                         y=CovidIndiaPlot["New_deaths"],
                         mode="lines",
                         line=dict(color='#8c564b', width=1.8),
                         name = "Daily cases"
                        )
             )
fig2.update_layout(title='Daily New Deaths in India',
                  autosize=True,
                  #plot_bgcolor="#FFFFFF",
#                   margin=dict(
#                         autoexpand=False,
#                         l=40,
#                         r=20,
#                         t=80,)
                        )
fig2.update_yaxes(range=[0,6000])
fig2.show()

### Outcome : Again data gathering from govt. is not proper shown by sudden spikes and lows in data
#### -------------------------------------------------------------------------------------------------------------------------------

In [18]:
# extract covid data from govt india website
def extract_india_covid_data():
    typ=[]
    value=[]
    data=requests.get("https://www.mohfw.gov.in/").text
    soup=BeautifulSoup(data,'lxml')
    dashboard=soup.find('section',class_='site-stats')
    d=dashboard.ul
    s=d.find_all('li')
    for i in s:
        x=i.find_all('strong',class_="mob-hide")
        types=re.search('"mob-hide">(.*)<span',str(x[0]))
        typ.append(types.group(1))
        values=re.search('"mob-hide">(.*)<span',str(x[1]))
        value.append(values.group(1))
    typ=[re.sub(' +','',i.replace('\xa0','')) for i in typ]
    value=[re.sub(' +','',i.replace('\xa0','')) for i in value]
    return pd.DataFrame([[i,int(j)] for i,j in zip(typ,value)],columns=['type','value'])

In [19]:
today=extract_india_covid_data()

In [20]:
fb=go.Figure([go.Bar(x=today['type'], 
                     y=today['value'], 
                     text=today['value'], 
                     textposition='auto',
                     marker_color='lightsalmon')])
fb.update_yaxes(range=[0,30000000])
fb.show()

# <center> State wise data analysis </center>

In [21]:
data=requests.get('https://api.rootnet.in/covid19-in/stats/latest').text

In [22]:
formatted_data=json.loads(data)

In [23]:
covid_india_state_data=formatted_data['data']['regional']

In [24]:
india_state_covid=pd.DataFrame.from_dict(covid_india_state_data)

In [25]:
india_state_covid=india_state_covid.astype({'loc':'string'})

In [26]:
india_state_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   loc                    36 non-null     string
 1   confirmedCasesIndian   36 non-null     int64 
 2   confirmedCasesForeign  36 non-null     int64 
 3   discharged             36 non-null     int64 
 4   deaths                 36 non-null     int64 
 5   totalConfirmed         36 non-null     int64 
dtypes: int64(5), string(1)
memory usage: 1.8 KB


In [27]:
india_state_covid_plot=india_state_covid[['loc','deaths','totalConfirmed']]

In [28]:
death_rate_comparision=india_state_covid_plot.sort_values(['deaths','totalConfirmed'],ascending=False)

In [29]:
fig3 = go.Figure()
fig3.add_trace(go.Bar(
    x=death_rate_comparision['loc'],
    y=death_rate_comparision['deaths'],
    name='Total Deaths',
    marker_color='indianred'
))

fig3.update_yaxes(range=[0,100000],tick0 = 0,dtick = 5000)
fig3.update_layout(autosize=False,
    width=1000,
    height=1000)


In [30]:
fig4 = go.Figure()
fig4.add_trace(go.Bar(
    x=death_rate_comparision['loc'],
    y=death_rate_comparision['totalConfirmed'],
    name='Active Cases',
    marker_color='lightsalmon'
))
fig4.update_yaxes(tick0 = 0,dtick = 1000000)
fig4.update_layout(autosize=False,
    width=1000,
    height=1000)
fig4.show()

### Looks Like Kerla is not reporting cases properly or it will take a very long time to reach peak 

In [31]:
death_rate_comparision['percentage']=round((death_rate_comparision['deaths']/death_rate_comparision['totalConfirmed'])*100,2)

In [32]:
death_rate_comparision.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 20 to 7
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   loc             36 non-null     string 
 1   deaths          36 non-null     int64  
 2   totalConfirmed  36 non-null     int64  
 3   percentage      36 non-null     float64
dtypes: float64(1), int64(2), string(1)
memory usage: 1.4 KB


In [33]:
death_rate_comparision.sort_values('percentage',ascending=False,inplace=True)

In [34]:
fig5 = go.Figure()
fig5.add_trace(go.Bar(
    x=death_rate_comparision['loc'],
    y=death_rate_comparision['percentage'],
    name='Active Cases',
    marker_color='rgb(0,128,128)'
))
fig5.update_yaxes(tick0 = 0,dtick = 0.2)
fig5.update_layout(autosize=False,
    width=1000,
    height=1000)
fig5.show()

### Looks like Pujbab is the worst hit state.

In [50]:
main=pd.DataFrame(columns=['loc','confirmedCasesIndian','confirmedCasesForeign','discharged','deaths','totalConfirmed','day'])
data=requests.get('https://api.rootnet.in/covid19-in/stats/history').text
formatted_data=json.loads(data)
covid_india_state_data_h=formatted_data['data']

for i in  range(1,len(covid_india_state_data_h)):
    tmp_data=covid_india_state_data_h[i]
    tmp_df=pd.DataFrame.from_dict(tmp_data['regional'])
    day=tmp_data['day']
    tmp_df['day']=day
    main=main.append(tmp_df)    

In [51]:
main.sort_values(['day','loc'])

,loc,confirmedCasesIndian,confirmedCasesForeign,discharged,deaths,totalConfirmed,day
0,Delhi,5,0,0,0,5,2020-03-11
1,Haryana,0,14,0,0,14,2020-03-11
9,Jammu and Kashmir,1,0,0,0,1,2020-03-11
2,Karnataka,4,0,0,0,4,2020-03-11
3,Kerala,17,0,0,0,17,2020-03-11
...,...,...,...,...,...,...,...
31,Telangana,516404,0,456620,2867,516404,2021-05-14
32,Tripura,39606,0,35146,426,39606,2021-05-14
34,Uttar Pradesh,1580979,1,1359676,16646,1580980,2021-05-14
33,Uttarakhand,271809,1,189261,4245,271810,2021-05-14


In [82]:
main[main['loc']=='Madhya Pradesh#']='Madhya Pradesh'
main[main['loc']=='Nagaland#']='Nagaland'
main[main['loc']=='Jharkhand#']='Jharkhand'

In [83]:
main[main['loc']=='Telangana***']='Telangana'
main[main['loc']=='Maharashtra***']='Maharashtra'
main[main['loc']=='Chandigarh***']='Chandigarh'
main[main['loc']=='Punjab***']='Punjab'
main[main['loc']=='Uttarakhand***']='Uttarakhand'

In [85]:
states=list(main['loc'].unique())

'Delhi'

In [ ]:
colors=[ aliceblue, antiquewhite, aqua, aquamarine, azure,
                beige, bisque, black, blanchedalmond, blue,
                blueviolet, brown, burlywood, cadetblue,
                chartreuse, chocolate, coral, cornflowerblue,
                cornsilk, crimson, cyan, darkblue, darkcyan,
                darkgoldenrod, darkgray, darkgrey, darkgreen,
                darkkhaki, darkmagenta, darkolivegreen, darkorange,
                darkorchid, darkred, darksalmon, darkseagreen,
                darkslateblue, darkslategray, darkslategrey,
                darkturquoise, darkviolet, deeppink, deepskyblue,
                dimgray, dimgrey, dodgerblue, firebrick,
                floralwhite, forestgreen, fuchsia, gainsboro,
                ghostwhite, gold, goldenrod, gray, grey, green,
                greenyellow, honeydew, hotpink, indianred, indigo,
                ivory, khaki, lavender, lavenderblush, lawngreen,
                lemonchiffon, lightblue, lightcoral, lightcyan,
                lightgoldenrodyellow, lightgray, lightgrey,
                lightgreen, lightpink, lightsalmon, lightseagreen,
                lightskyblue, lightslategray, lightslategrey,
                lightsteelblue, lightyellow, lime, limegreen,
                linen, magenta, maroon, mediumaquamarine,
                mediumblue, mediumorchid, mediumpurple,
                mediumseagreen, mediumslateblue, mediumspringgreen,
                mediumturquoise, mediumvioletred, midnightblue,
                mintcream, mistyrose, moccasin, navajowhite, navy,
                oldlace, olive, olivedrab, orange, orangered,
                orchid, palegoldenrod, palegreen, paleturquoise,
                palevioletred, papayawhip, peachpuff, peru, pink,
                plum, powderblue, purple, red, rosybrown,
                royalblue, saddlebrown, salmon, sandybrown,
                seagreen, seashell, sienna, silver, skyblue,
                slateblue, slategray, slategrey, snow, springgreen,
                steelblue, tan, teal, thistle, tomato, turquoise,
                violet, wheat, white, whitesmoke, yellow,
                yellowgreen]

In [163]:
fig6 = go.Figure()

for i in range(0,len(states)):
    fig6.add_trace(go.Scatter(x=main[main['loc']==states[i]]['day'], y=main[main['loc']==states[i]]['confirmedCasesIndian'], 
                              mode='lines',
        name=states[i],
        connectgaps=True
    ))
fig6.update_layout(autosize=False,
    width=1000,
    height=1000,
    showlegend=False)
fig6.update_yaxes(range=[0,7000000],tick0=1000000,dtick="400000")
fig6.show()
